In [2]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
num_time_shifts = 100
num_velocities = 32
min_time_shift = 0.01

In [3]:
data = np.load('test_history.npy', allow_pickle=True)
inst = np.load('test_instruments.npy', allow_pickle=True)

In [4]:
def data_to_midi(data, instrument_number):
    instrument=pretty_midi.Instrument(program=instrument_number,name=pretty_midi.program_to_instrument_name(instrument_number))
    curr_time = 0
    curr_velocity = 96
    #start time and velocity of the current note being played
    note_start = {}
    note_velocity = {}
    note_same = {}
    for i, idx in enumerate(data):
        if idx < num_notes:
            #note on event
            if idx in note_start:
                note_same[idx] += 1
                continue
            note_start[idx] = curr_time
            note_velocity[idx] = curr_velocity
            note_same[idx] = 1
        elif idx < 2*num_notes:
            #note off event
            curr_note = idx - num_notes
            if curr_note not in note_start:
                continue
                #raise ValueError("note" + str(idx-num_notes) + "has not been started!")
            note_same[curr_note] -= 1
            if note_same[curr_note] == 0:
                note=pretty_midi.Note(velocity=note_velocity[curr_note], pitch=curr_note, start=note_start[curr_note], end=curr_time)
                instrument.notes.append(note)
                del note_start[curr_note]
                del note_velocity[curr_note]
                del note_same[curr_note]
        elif idx < 2*num_notes + num_velocities:
            #velocity event
            #reversing the quantization in preprocess operation
            curr_velocity=int((idx-2*num_notes)*128/num_velocities)
        elif idx < 2*num_notes + num_time_shifts + num_velocities:
            #time shift event
            curr_time += (idx - 2*num_notes - num_velocities + 1)*min_time_shift
    return instrument
    

In [5]:
recording = pretty_midi.PrettyMIDI()
for i,j in zip(data, inst):
    recording.instruments.append(data_to_midi(i,j))

In [7]:
recording.write("test_midi.mid")

In [30]:
file = open( "preprocessed_data_fnames.p", "rb" )
res = pickle.load(file)
file.close()